In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:15 http://ppa

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [4]:
# Read in the Review dataset as a smaller DataFrame (too large and was failing)
# import random 
from random import sample

df_sample = df.sample(fraction=0.5)
df_sample.count()

453316

In [5]:
# Create the vine_table. DataFrame
vine_df = df_sample.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|R14YLXA56NP51I|          5|            1|          1|   N|                N|
|R3H53KLLC210XI|          4|            0|          0|   N|                Y|
|R3J44DPP12OTLJ|          5|            0|          0|   N|                Y|
|R2HUWDNW62FOL3|          1|            0|          0|   N|                Y|
|R1EPGPK802HBSK|          5|            0|          0|   N|                Y|
| RJIPC1X7LJ1RF|          5|            0|          0|   N|     

In [14]:
# Step 1: Filter data to create DataFrame for results with over 20 votes
votefilter_df = vine_df.filter(vine_df.total_votes > 20)
votefilter_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|  RMRD6SMF2AUQ|          3|            4|         21|   N|                N|
| RMPCXKWX3T57Y|          1|            1|         73|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
| RC6Y8GYIQZ8YU|          3|           72|         85|   N|                Y|
|R3PRNAQVQBB9M6|          1|           21|         30|   N|                Y|
|R3JNDLZ3KACRLG|          5|           29|         33|   N|                N|
|R1PVVYYY0Q1HZ0|          5|           21|         21|   N|                Y|
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|     

In [22]:
# Step 2: Filter DataFrame from Step 1 (Create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.)

helpful_df = votefilter_df.filter(votefilter_df.helpful_votes / votefilter_df.total_votes >= 0.5)
helpful_df.show()
helpful_df.count()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
| RC6Y8GYIQZ8YU|          3|           72|         85|   N|                Y|
|R3PRNAQVQBB9M6|          1|           21|         30|   N|                Y|
|R3JNDLZ3KACRLG|          5|           29|         33|   N|                N|
|R1PVVYYY0Q1HZ0|          5|           21|         21|   N|                Y|
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|
|R1G9N0CW2982PA|          5|           21|         24|   N|                Y|
|R2N8ZFHQBWHMDL|          2|           26|         28|   N|     

6796

In [21]:
# Step 3: Filter DataFrame from Step 2 (Filter out all Non-Vine Reviews)

validvine_df = helpful_df.filter(helpful_df.vine == "Y")
validvine_df.show()
validvine_df.count()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|
|R34DJ1R8AEU0SG|          5|           29|         35|   Y|                N|
|R25P5CXK5L9RHF|          5|          146|        161|   Y|                N|
| RKYLHZL7EPELX|          4|           19|         25|   Y|                N|
|R1U13EKGQD3ZE6|          5|           22|         25|   Y|                N|
| RYW05F1MUEF01|          5|           87|        102|   Y|                N|
|R13PPTBWWV10Q6|          5|           19|         25|   Y|                N|
|R3QQG4HUB214TM|          5|          111|        128|   Y|                N|
|R22CYBPLKWDK6F|          5|           47|         54|   Y|                N|
|R28T5ISQ8AA7E5|          5|           40|         42|   Y|     

35

In [20]:
# Step 4. Filter DataFrame from Step 2 (Filter out all Vine Reviews)
nonVine_df = helpful_df.filter(helpful_df.vine == "N")
nonVine_df.show()
nonVine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
| RC6Y8GYIQZ8YU|          3|           72|         85|   N|                Y|
|R3PRNAQVQBB9M6|          1|           21|         30|   N|                Y|
|R3JNDLZ3KACRLG|          5|           29|         33|   N|                N|
|R1PVVYYY0Q1HZ0|          5|           21|         21|   N|                Y|
|R1G9N0CW2982PA|          5|           21|         24|   N|                Y|
|R2N8ZFHQBWHMDL|          2|           26|         28|   N|                Y|
|R32L3HQ3OONLPQ|          4|           51|         54|   N|     

6761

In [24]:
# Step 5 - Total Number of Reviews
helpful_df.count()



6796

In [26]:
# Step 5 - Total Number of 5-star Reviews
fivestar = helpful_df.filter(helpful_df.star_rating == 5).count()
fivestar

3852

In [41]:
# # Step 5 - % of 5 star reviews for paid vs unpaid 
# Paid (Vine) 5 Star reviews
paid_df = validvine_df.filter(validvine_df.star_rating == 5).count()/fivestar * 100
paid_df



0.5451713395638629

In [40]:
# # Step 5 - Unpaid (nonVine) 5 Star Reviews
unpaid_df = nonVine_df.filter(nonVine_df.star_rating == 5).count()/fivestar * 100
funpaid_df

99.45482866043614